# Step 1 - Create or retrieve your Azure ML Workspace #

In [ ]:
# Provide the Subscription ID of your existing Azure subscription
subscription_id = "xxx-xxx-xxx"

# Provide values for the Resource Group and Workspace that will be created
resource_group = "service-labs"
workspace_name = "service-labs-ws"
workspace_region = "eastus"  # eastus, westcentralus, southeastasia, australiaeast, westeurope

In [ ]:
import azureml
from azureml.core import Workspace

print("SDK Version:", azureml.core.VERSION)

# By using the exist_ok param, if the worskpace already exists we get a reference to the existing workspace
ws = Workspace.create(
    name = workspace_name,
    subscription_id = subscription_id,
    resource_group = resource_group, 
    location = workspace_region,
    exist_ok = True)

print("Workspace Provisioning complete.")

# Step 2 - Provision an IoT Hub and an IoT Edge Device #

If you are using **Visual Studio Code**, please ensure that Azure CLI is installed on your local machine. You can find the instructions on installing Azure CLI at: https://docs.microsoft.com/en-us/cli/azure/install-azure-cli?view=azure-cli-latest

**Log in Azure CLI**

In [ ]:
!az login

**Set the subcription**

In [ ]:
!az account set --subscription $subscription_id

**You will need to use the Azure IoT extension to the Azure CLI**

In [ ]:
!az extension add --name azure-cli-iot-ext

### Provision an IoT Hub

In [ ]:
# Azure IoT hub name
iot_hub_name="sl-iot-hub"
# Pick a name or ID for your IoT Edge device
iot_device_id="slEdgeDevice"

**It will take 2-3 minutes to create the IoT Hub**

In [ ]:
# Create a new IoT hub
!az iot hub create --resource-group $resource_group --name $iot_hub_name --sku S1

In [ ]:
# Register an IoT Edge device (create a new entry in the Iot Hub)
!az iot hub device-identity create --hub-name $iot_hub_name --device-id $iot_device_id --edge-enabled

In [ ]:
# Get the connection string that you will need to enter in the IoT Edge device
!az iot hub device-identity show-connection-string --device-id $iot_device_id --hub-name $iot_hub_name

### Provision an IoT Edge Device

In [ ]:
# Name of the IoT Edge device VM
iot_edge_device_name = "EdgeVM"
# Image for the IoT Edge device VM
device_vm = "microsoft_iot_edge:iot_edge_vm_ubuntu:ubuntu_1604_edgeruntimeonly:latest"
# User name
vm_username = "azureuser"

In [ ]:
!az vm image terms accept --urn $device_vm

**It will take 2-3 minutes to create the IoT Edge VM**

From the output below, save the **publicIpAddress** of the IoT edge device VM for later use.

In [ ]:
!az vm create --resource-group $resource_group --name $iot_edge_device_name --image $device_vm --admin-username $vm_username --generate-ssh-keys

## Enter the connection string into your IoT Edge device ##

Copy the connection string from the `show-connection-string` command above. Replace the following with your connection string within single quotes in the cell below and then run the cell.

'HostName=sl-iot-hub.azure-devices.net;DeviceId=slEdgeDevice;SharedAccessKey=5oOVh9fFGR2iafdzfrKLvyKJto6cm30XsaLegWu8jrs='

In [ ]:
!az vm run-command invoke -g $resource_group -n $iot_edge_device_name --command-id RunShellScript --script "/etc/iotedge/configedge.sh 'HostName=sl-iot-hub.azure-devices.net;DeviceId=slEdgeDevice;SharedAccessKey=5oOVh9fFGR2iafdzfrKLvyKJto6cm30XsaLegWu8jrs='"

# Step 3 - Build the ContainerImage for the IoT Edge Module #

## Register the pretrained Anomaly Detection Model

In [ ]:
from azureml.core.model import Model

model = Model.register(model_path = "./model.pkl",
                       model_name = "anomaly-detector", 
                       description = "Anomaly detector service", 
                       workspace = ws)

print("Model Name: {} \nModel Description: {} \nModel Version: {}".format(model.name, 
                                                                                model.description, model.version))

## Package Model

You can review the accompanying scoring script ` iot_score.py` that will be used to make real-time inferences using the registered model.

**Create dependencies to include in the environment**

In [ ]:
print("Creating conda dependencies file locally...")
from azureml.core.conda_dependencies import CondaDependencies 
conda_packages = ['numpy==1.16.2', 'pandas==0.23.4', 'scikit-learn==0.20.3']
pip_packages = ['azureml-defaults']
mycondaenv = CondaDependencies.create(conda_packages=conda_packages, pip_packages=pip_packages)

conda_file = "myenv.yml"
with open(conda_file, 'w') as f:
    f.write(mycondaenv.serialize_to_string())

*The next cell may take 5-10 minutes to complete*

In [ ]:
from azureml.core.image import ContainerImage, Image

runtime = "python" 
driver_file = "iot_score.py"

image_config = ContainerImage.image_configuration(execution_script = driver_file,
                                                  runtime = runtime,
                                                  conda_file = conda_file)

image = Image.create(name = "iotimage",
                     models = [model],
                     image_config = image_config, 
                     workspace = ws)

image.wait_for_creation(show_output = True)

# Step 4 - Deploy the modules #

Create a deployment.json file that contains the modules you want to deploy to the device and the routes. Then push this file to the IoT Hub, which will then send it to the IoT Edge device. The IoT Edge agent will then pull the Docker images and run them.

**Get your container details**

In [ ]:
container_reg = image.image_location
reg_name=container_reg.split("/")[-2]
reg_name=reg_name.split(".")[-3]
container_url = "\"" + image.image_location + "\","
subscription_id = ws.subscription_id
print('{}'.format(image.image_location))
print('{}'.format(reg_name))
print('{}'.format(subscription_id))
from azure.mgmt.containerregistry import ContainerRegistryManagementClient
from azure.mgmt import containerregistry
client = ContainerRegistryManagementClient(ws._auth, subscription_id)
result= client.registries.list_credentials(resource_group, reg_name, custom_headers=None, raw=False)
username = result.username
password = result.passwords[0].value

**Update the deployment template**

In [ ]:
module_name = "machinelearningmodule"
file = open('deployment-template.json')
contents = file.read()
contents = contents.replace('__MODULE_NAME', module_name)
contents = contents.replace('__REGISTRY_NAME', reg_name)
contents = contents.replace('__REGISTRY_USER_NAME', username)
contents = contents.replace('__REGISTRY_PASSWORD', password)
contents = contents.replace('__REGISTRY_IMAGE_LOCATION', image.image_location)
with open('./deployment.json', 'wt', encoding='utf-8') as output_file:
    output_file.write(contents)

## Push the deployment JSON to the IOT Hub

In [ ]:
!az iot edge set-modules --device-id $iot_device_id --hub-name $iot_hub_name --content deployment.json

# Step 5 - Examine the scored messages #

1. From within Azure Portal, navigate to your resource group and select your IoT hub: **sl-iot-hub**. 

2. From the `Automatic Device Management` section of your IoT hub, select **IoTEdge, slEdgeDevice**.

3. In the `slEdgeDevice` edge device page, confirm that the two added modules **tempSensor** and **machinelearningmodule** are in **running** status. It may take about 5-10 minutes for the two new modules to appear and start running. Once you see a `RUNTIME STATUS` of `running` for all modules you can proceed.

4. From your local machine open a new `Command Prompt` window.

5. Connect to your VM via SSH, using the `publicIpAddress` you acquired previously.
    ```
    ssh azureuser@{publicIpAddress}
    ```
6. View the anomaly detection scored messages being sent by the `machinelearningmodule` by running the following command.
    ```
    iotedge logs machinelearningmodule -f
    ```